In [2]:
import pandas as pd
data_2013 = pd.read_csv('MERGED2013_14_PP.csv',encoding="utf-8-sig")

#Question 1:
q1_related_cols = ["UNITID","OPEID","ICLEVEL","SAT_AVG","UGDS"]
df_q1 = data_2013[q1_related_cols]
df_q1 = df_q1.round(10)
df_q1 = df_q1.dropna()
df_q1 = df_q1[df_q1['ICLEVEL']==1]
df_q1['New_admitted'] = df_q1['UGDS'].astype(float)/4.0
df_q1['weighted']= df_q1['SAT_AVG'].astype(float)*df_q1['New_admitted'].astype(float)/df_q1['New_admitted'].astype(float).sum()
avg_sat = df_q1['weighted'].astype(float).sum()
print(avg_sat)

#Question 2:
q2_related_cols = ["UNITID",'OPEID','UGDS','ICLEVEL','LO_INC_COMP_ORIG_YR4_RT','LO_INC_COMP_4YR_TRANS_YR4_RT','MD_INC_COMP_ORIG_YR4_RT',
                   'MD_INC_COMP_4YR_TRANS_YR4_RT','HI_INC_COMP_ORIG_YR4_RT',
                   'HI_INC_COMP_4YR_TRANS_YR4_RT']
df_q2 = data_2013[q2_related_cols]
df_q2 = df_q2[df_q2['ICLEVEL']==1]
reduced_cols = ['LO_INC_COMP_ORIG_YR4_RT','LO_INC_COMP_4YR_TRANS_YR4_RT','MD_INC_COMP_ORIG_YR4_RT',
                'MD_INC_COMP_4YR_TRANS_YR4_RT','HI_INC_COMP_ORIG_YR4_RT','HI_INC_COMP_4YR_TRANS_YR4_RT']
df_q2 = df_q2.dropna()
for col in reduced_cols:
    condition = (df_q2[col]!="PrivacySuppressed")
    df_q2 = df_q2[condition]
    
df_q2['lo_within_4yrs'] = df_q2['LO_INC_COMP_ORIG_YR4_RT'].astype(float)+df_q2['LO_INC_COMP_4YR_TRANS_YR4_RT'].astype(float)
df_q2['hi_within_4yrs'] = df_q2['HI_INC_COMP_ORIG_YR4_RT'].astype(float)+df_q2['HI_INC_COMP_4YR_TRANS_YR4_RT'].astype(float)
diff = df_q2['hi_within_4yrs'].mean() - df_q2['lo_within_4yrs'].mean()
print(round(diff,10))

#Question 3:
q3_related_cols =["UNITID",'OPEID','UGDS_WHITE','UGDS_BLACK','UGDS_HISP','UGDS_ASIAN','UGDS_AIAN','UGDS_NHPI']
df_q3 = data_2013[q3_related_cols]
filtered_cols = ['UGDS_WHITE','UGDS_BLACK','UGDS_HISP','UGDS_ASIAN','UGDS_AIAN','UGDS_NHPI']
df_q3 = df_q3[df_q3[filtered_cols].notnull().any(axis=1)]
df_q3_new = df_q3[filtered_cols]
df_q3_new['Max'] = df_q3_new.max(axis=1)
df_q3_new['Min'] = df_q3_new.min(axis=1)
df_q3 = pd.concat([df_q3, df_q3_new[['Max','Min']]],axis=1)
df_q3['Max-Min'] = df_q3['Max'] - df_q3['Min']
df_q3_final = df_q3[df_q3['Max-Min']>0]
min_diff = df_q3_final.min()[-1]
print(round(min_diff,10))

#Question 4:
data_2014 = pd.read_csv('MERGED2014_15_PP.csv',encoding="utf-8-sig")
q4_related_cols = ["UNITID",'OPEID',"REGION","LOCALE"]
df_q4 = data_2014[q4_related_cols]
df_q4 = df_q4.dropna()
df_q4 = df_q4[df_q4['LOCALE']>0]
def is_city(code):
    if code < 20:
        return "City"
    return "Not City"
df_q4['City'] = df_q4['LOCALE'].apply(is_city)
region_lst = range(1,10)
city_percentage_region = {}
for region in region_lst:
    df_region = df_q4[df_q4['REGION']==region]
    df_city = df_region[df_region['City']=='City']
    percentage = float(len(df_city))/float(len(df_region))
    city_percentage_region[region] = percentage
maximum = max(city_percentage_region.values())
print(round(maximum,10))

#Question 5:
q5_related_cols = ["UNITID","OPEID","SAT_AVG","ENRL_ORIG_YR2_RT"]
df_q5 = data_2013[q5_related_cols]
df_q5 = df_q5.dropna()
df_q5 = df_q5[df_q5['ENRL_ORIG_YR2_RT']!="PrivacySuppressed"]
df_q5['ENRL_ORIG_YR2_RT'] = df_q5['ENRL_ORIG_YR2_RT'].astype(float)
correlation = df_q5.corr()['ENRL_ORIG_YR2_RT'][2]
print(round(correlation,10))

#Question 6:
import scipy.stats
import math
test = scipy.stats.ttest_ind(df_q2['lo_within_4yrs'],df_q2['hi_within_4yrs'])
p_value = test[1]
logp_value = - math.log10(p_value)
print(p_value,round(logp_value,10))

#Question 7:
files = ['MERGED2001_02_PP.csv','MERGED2002_03_PP.csv','MERGED2003_04_PP.csv',
         'MERGED2004_05_PP.csv','MERGED2005_06_PP.csv','MERGED2006_07_PP.csv',
         'MERGED2007_08_PP.csv','MERGED2008_09_PP.csv','MERGED2009_10_PP.csv','MERGED2010_11_PP.csv']
q7_related_cols = ["UNITID","UGDS","UGDS_WOMEN"]
UGDS_lst = []
women_share = []
simple_avg = []
weighted_avg = []
for f in files:
    data = pd.read_csv(f,encoding="utf-8-sig")
    df = data[q7_related_cols]
    df = df.dropna()
    df['weighted_women_share'] = df['UGDS']*df['UGDS_WOMEN']
    simple_avg.append(df['UGDS_WOMEN'].mean())
    weighted_avg.append(df['weighted_women_share'].sum()/df['UGDS'].sum())
    UGDS_lst.append(df['UGDS'].sum())
    women_share.append(df['weighted_women_share'].sum())
print(round(sum(women_share)/sum(UGDS_lst),10))

1097.00795562
0.1376321945
0.0275


C:\tool\Anaconda2\lib\site-packages\ipykernel\__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\tool\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (6,9,1379,1380,1381,1382,1383,1384,1385,1386,1387,1388,1389,1390,1391,1392,1393,1394,1395,1396,1397,1398,1399,1400,1401,1402,1403,1404,1405,1406,1407,1411,1425,1427,1503,1517,1529,1530,1532,1537,1540,1541,1542,1575,1576,1577,1578,1579,1580,1581,1582,1583,1584,1585,1586,1587,1588,1589,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599,1600,1601,1602,1605,1606,1609,1610,1613,1614,1615,1725,1726,1727,1728,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0.6233933162
0.6618369648
(2.1082863875123491e-78, 77.6760703953)


C:\tool\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\tool\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (1,6,442,451,598,607,611,620,624,633,789,802,806,815,893,906,971,984,997,1153,1166,1407,1408,1411,1425,1431,1432,1433,1437,1438,1439,1440,1445,1446,1447,1451,1452,1453,1454,1459,1460,1461,1465,1466,1467,1468,1473,1474,1475,1479,1480,1481,1482,1487,1488,1489,1501,1502,1537,1538,1539,1540,1541,1542,1603,1606,1609,1610,1611,1613,1614,1615,1616,1688,1689,1690,1691,1692,1729) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\tool\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (1,1729) have mixed types. Specify dtype option on im

0.5732395864
